In [2]:
from __future__ import print_function
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import LSTM, GRU, Embedding, Dense, Flatten, Bidirectional, Dense, Dropout, Activation, BatchNormalization
import numpy as np
import pandas as pd

# Import the dataset
df = pd.read_csv('D3_data.csv', header=None)
df.columns =['Index', 'Url', 'Result']
len(df)

126077

In [5]:
urls = {}
for index, row in df.iterrows():
    urls[row['Url']] = row['Result']

In [6]:
samples = []
labels = []
for k, v in urls.items():
    samples.append(k)
    labels.append(v)

In [7]:
# Preprocess data for training.
max_chars = 20000
maxlen = 128

tokenizer = Tokenizer(num_words=max_chars, char_level=True)
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 67 unique tokens.
Shape of data tensor: (125846, 128)
Shape of label tensor: (125846,)


In [8]:
# Divide data between training, cross-validation, and test data.
training_samples = int(len(samples) * 0.95)
validation_samples = int(len(labels) * 0.05)
print(training_samples, validation_samples)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x = data[:training_samples]
y = labels[:training_samples]
x_test = data[training_samples: training_samples + validation_samples]
y_test = labels[training_samples: training_samples + validation_samples]

119553 6292


In [11]:
# Define callbacks for Keras.
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
    filepath='url_lstm.h5',
    monitor='val_loss',
    save_best_only=True
    ),
    keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0,
    patience=2, 
    mode='auto',
    baseline=None,
    )
]

num_chars = len(tokenizer.word_index)+1

embedding_vector_length = 128


In [12]:
# Create model for training.
model = Sequential()
model.add(Embedding(num_chars, embedding_vector_length, input_length=maxlen))
model.add(Bidirectional(LSTM(256, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
model.add(Bidirectional(LSTM(256, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 128, 128)          8704      
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 512)          788480    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 512)          1574912   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               656384    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 3,028,737
Trainable params: 3,028,737
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train.
model.fit(x, y,
        epochs=10,
        batch_size=1200,
        callbacks=callbacks_list,
        validation_split=0.20,
        shuffle=True
        )

# Evaluate model on test data.
score, acc = model.evaluate(x_test, y_test, verbose=1, batch_size=1024)

print("Model Accuracy: {:0.2f}%".format(acc * 100))

In [9]:
model = load_model('url_lstm.h5')

# Evaluate model on test data.
score, acc = model.evaluate(x_test, y_test, verbose=1, batch_size=1024)

print("Model Accuracy: {:0.2f}%".format(acc * 100))

7/7 [==============================] - 8s 1s/step - loss: 0.2007 - accuracy: 0.9272
Model Accuracy: 92.72%
